# A. Instantiate Data Context

In [ ]:
# Create a data context
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='./')

In [ ]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'csv-data-jan'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'sales-january'
path_to_data = '/content/P2M3_Fajar_Dawud_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# B. Create an Expectation Suite

In [ ]:
# Creat an expectation suite
expectation_suite_name = 'expectation-sales-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head(20)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,operating_margin,sales_method
0,Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,50.0,1200,600000.0,300000.0,0.50,In-store
1,Foot Locker,1185732,2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,500000.0,150000.0,0.30,In-store
2,Foot Locker,1185732,2020-01-03,Northeast,New York,New York,Women's Street Footwear,40.0,1000,400000.0,140000.0,0.35,In-store
3,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,382500.0,133875.0,0.35,In-store
4,Foot Locker,1185732,2020-01-05,Northeast,New York,New York,Men's Apparel,60.0,900,540000.0,162000.0,0.30,In-store
5,Foot Locker,1185732,2020-01-06,Northeast,New York,New York,Women's Apparel,50.0,1000,500000.0,125000.0,0.25,In-store
6,Foot Locker,1185732,2020-01-07,Northeast,New York,New York,Men's Street Footwear,50.0,1250,625000.0,312500.0,0.50,In-store
7,Foot Locker,1185732,2020-01-08,Northeast,New York,New York,Men's Athletic Footwear,50.0,900,450000.0,135000.0,0.30,Outlet
8,Foot Locker,1185732,2020-01-21,Northeast,New York,New York,Women's Street Footwear,40.0,950,380000.0,133000.0,0.35,Outlet
9,Foot Locker,1185732,2020-01-22,Northeast,New York,New York,Women's Athletic Footwear,45.0,825,371250.0,129937.5,0.35,Outlet


# C. Expectations

### 1.`expect_column_values_to_be_unique`
Kolom baru: invoice_id (kombinasi retailer_id, invoice_date, dan product, units_sold, total_sales)

In [ ]:
# Tambah kolom invoice_id ke dataframe
df = validator.active_batch.data.dataframe.copy()

df["invoice_id"] = (
    df["retailer_id"].astype(str) + "_" +
    pd.to_datetime(df["invoice_date"]).dt.strftime("%Y%m%d") + "_" +
    df["product"].astype(str) + "_" +
    df["units_sold"].astype(str) + "_" +
    df["total_sales"].astype(str) + "_" +
    df.index.astype(str)
)


# Buat ulang asset dari dataframe yang sudah dimodifikasi
new_datasource = context.sources.add_pandas("temporaryy_data")
new_asset = new_datasource.add_dataframe_asset("with_invoice_id", dataframe=df)

# Build batch dan validator baru
batch_request_new = new_asset.build_batch_request()
validator = context.get_validator(
    batch_request=batch_request_new,
    expectation_suite_name=expectation_suite_name
)

# Sekarang validator udah tahu soal invoice_id
validator.expect_column_values_to_be_unique("invoice_id")


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_unique",
    "kwargs": {
      "column": "invoice_id",
      "batch_id": "temporaryy_data-with_invoice_id"
    },
    "meta": {}
  },
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### 2.`expect_to_be_between_min_value_and_max_value`
dengan menggunakan kolom jumlah barang yang terjual (`units_sold`), saya ingin melihat rentang nilai minimum dan maximum nya

In [ ]:
validator.expect_column_values_to_be_between("units_sold", min_value=0, max_value=1275)


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "min_value": 0,
      "max_value": 1275,
      "column": "units_sold",
      "batch_id": "temporaryy_data-with_invoice_id"
    },
    "meta": {}
  },
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### 3.`expect_column_values_to_be_in_set`

In [ ]:
validator.expect_column_values_to_be_in_set("sales_method", ["Outlet", "Online", "In-store"])


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_in_set",
    "kwargs": {
      "column": "sales_method",
      "value_set": [
        "Outlet",
        "Online",
        "In-store"
      ],
      "batch_id": "temporaryy_data-with_invoice_id"
    },
    "meta": {}
  },
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### 4.`expect_column_values_to_be_in_type_list`

In [ ]:
validator.expect_column_values_to_be_of_type("price_per_unit", "float64")


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_of_type",
    "kwargs": {
      "column": "price_per_unit",
      "type_": "float64",
      "batch_id": "temporaryy_data-with_invoice_id"
    },
    "meta": {}
  },
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### 5`expect_column_values_to_be_match_regex`.

In [ ]:
validator.expect_column_values_to_match_regex("product", r"^[A-Za-z0-9\s&'\-]+$")


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_match_regex",
    "kwargs": {
      "column": "product",
      "regex": "^[A-Za-z0-9\\s&'\\-]+$",
      "batch_id": "temporaryy_data-with_invoice_id"
    },
    "meta": {}
  },
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### 6.`expectation_column_median_to_be_between`

In [ ]:
validator.expect_column_median_to_be_between("total_sales", min_value=9550, max_value=9650)


Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_median_to_be_between",
    "kwargs": {
      "min_value": 9550,
      "max_value": 9650,
      "column": "total_sales",
      "batch_id": "temporaryy_data-with_invoice_id"
    },
    "meta": {}
  },
  "result": {
    "observed_value": 9576.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### 7.`expectation_column_pair_values_A_to_be_greater_than_B`

In [ ]:
validator.expect_column_pair_values_A_to_be_greater_than_B("price_per_unit", "operating_margin")


Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_pair_values_a_to_be_greater_than_b",
    "kwargs": {
      "column_A": "price_per_unit",
      "column_B": "operating_margin",
      "batch_id": "temporaryy_data-with_invoice_id"
    },
    "meta": {}
  },
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}